## 1. Subscribe to the model package

To subscribe to the model package:
1. Open the model package listing page: [NCIT Clinical Terminology Mapper](https://aws.amazon.com/marketplace/pp/prodview-cnhtyygml2lms)
1. On the AWS Marketplace listing, click on the **Continue to subscribe** button.
1. On the **Subscribe to this software** page, review and click on **"Accept Offer"** if you and your organization agrees with EULA, pricing, and support terms. 
1. Once you click on **Continue to configuration button** and then choose a **region**, you will see a **Product Arn** displayed. This is the model package ARN that you need to specify while creating a deployable model using Boto3. Copy the ARN corresponding to your region and specify the same in the following cell.

## NCIT Clinical Terminology Mapper

- **Model**: `ncit_vdb_resolver`
- **Model Description**: This pretrained pipeline extracts oncological entities from clinical texts and maps them to their corresponding National Cancer Institute Thesaurus (NCIt) codes.

In [1]:
model_package_arn = "<Customer to specify Model package ARN corresponding to their AWS region>"

In [ ]:
import json
import os
import boto3
import pandas as pd
import sagemaker as sage
from sagemaker import ModelPackage
from sagemaker import get_execution_role
from IPython.display import display
from urllib.parse import urlparse

In [3]:
sagemaker_session = sage.Session()
s3_bucket = sagemaker_session.default_bucket()
region = sagemaker_session.boto_region_name
account_id = boto3.client("sts").get_caller_identity().get("Account")
role = get_execution_role()

sagemaker = boto3.client("sagemaker")
s3_client = sagemaker_session.boto_session.client("s3")
ecr = boto3.client("ecr")
sm_runtime = boto3.client("sagemaker-runtime")

# Set display options
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [4]:
model_name = "ncit-vdb-resolver"

real_time_inference_instance_type = "ml.m4.xlarge"
batch_transform_inference_instance_type = "ml.m4.2xlarge"

## 2. Create a deployable model from the model package.

In [5]:
model = ModelPackage(
    role=role, 
    model_package_arn=model_package_arn,
    sagemaker_session=sagemaker_session,
)

### Input Format

To use the model, you need to provide input in one of the following supported formats:

#### JSON Format

Provide input as JSON. We support two variations within this format:

1. **Array of Text Documents**: 
   Use an array containing multiple text documents. Each element represents a separate text document.

   ```json
   {
       "text": [
           "Text document 1",
           "Text document 2",
           ...
       ]
   }

    ```

2. **Single Text Document**:
   Provide a single text document as a string.


   ```json
    {
        "text": "Single text document"
    }
   ```

#### JSON Lines (JSONL) Format

Provide input in JSON Lines format, where each line is a JSON object representing a text document.

```
{"text": "Text document 1"}
{"text": "Text document 2"}
```

## 3. Create an endpoint and perform real-time inference

If you want to understand how real-time inference with Amazon SageMaker works, see [Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-hosting.html).

### A. Deploy the SageMaker model to an endpoint

In [6]:
predictor = model.deploy(
    initial_instance_count=1,
    instance_type=real_time_inference_instance_type, 
    endpoint_name=model_name,
)

------------!

Once endpoint has been created, you would be able to perform real-time inference.

In [7]:
def invoke_realtime_endpoint(record, content_type="application/json", accept="application/json"):
    response = sm_runtime.invoke_endpoint(
        EndpointName=model_name,
        ContentType=content_type,
        Accept=accept,
        Body=json.dumps(record) if content_type == "application/json" else record,
    )

    response_body = response["Body"].read().decode("utf-8")

    if accept == "application/json":
        return json.loads(response_body)
    elif accept == "application/jsonlines":
        return response_body
    else:
        raise ValueError(f"Unsupported accept type: {accept}")

### Initial Setup

In [8]:
docs = [
    '''A 65-year-old woman had a history of debulking surgery, bilateral oophorectomy with omentectomy, total hysterectomy with radical pelvic lymph nodes dissection due to ovarian carcinoma (mucinous-type carcinoma, stage Ic) 1 year ago. Patient's medical compliance was poor and failed to complete her chemotherapy (cyclophosphamide 750 mg/m2, carboplatin 300 mg/m2). Recently, she noted a palpable right breast mass, 15 cm in size which nearly occupied the whole right breast in 2 months. Core needle biopsy revealed metaplastic carcinoma.''',
    '''She underwent a computed tomography (CT) scan of the abdomen and pelvis, which showed a complex ovarian mass. A Pap smear performed one month later was positive for atypical glandular cells suspicious for adenocarcinoma. The pathologic specimen showed extension of the tumor throughout the fallopian tubes, appendix, omentum, and 5 out of 5 enlarged lymph nodes. The final pathologic diagnosis of the tumor was stage IIIC papillary serous ovarian adenocarcinoma. Two months later, the patient was diagnosed with lung metastases.'''
]

sample_text = '''There was no differentiated component characteristic of adenocarcinoma, acinar cell carcinoma or hepatoid carcinoma of the pancreas. The surgical margins were negative for neoplastic infiltration. There was no evidence of perineural invasion. No lymph node metastasis was shown.'''

### JSON

In [9]:
input_json_data = {"text": sample_text}
response_json = invoke_realtime_endpoint(input_json_data, content_type="application/json", accept="application/json")
pd.DataFrame(response_json["predictions"][0])

,begin,end,ner_chunk,ner_label,ner_confidence,concept_code,resolution,score,all_codes,all_resolutions,all_score
0,56,69,adenocarcinoma,Cancer_dx,None,C5105,colorectal adenocarcinoma,0.903468,"[C5105, C4349, C4349, C4349, C3512]","[colorectal adenocarcinoma, colon adenocarcinoma, adenocarcinoma of the colon, adenocarcinoma of colon, lung adenocarcinoma]","[0.9034679532051086, 0.894524097442627, 0.8823219537734985, 0.8813044428825378, 0.8766085505485535]"
1,72,92,acinar cell carcinoma,Cancer_dx,None,C3768,acinar cell carcinoma,1.000000,"[C3768, C3768, C3768, C4197, C4197]","[acinar cell carcinoma, acinar carcinoma, acinar cell adenocarcinoma, acinar cell neoplasms, acinar cell neoplasm]","[1.0000001192092896, 0.9744840860366821, 0.958342432975769, 0.9396045804023743, 0.9360271692276001]"
2,97,104,hepatoid,Histological_Type,0.9959,C66950,hepatoid carcinoma,0.840701,"[C66950, C66950, C95465, C13308, C95747]","[hepatoid carcinoma, hepatoid adenocarcinoma, pancreatic hepatoid carcinoma, hepatic, gastric hepatoid adenocarcinoma]","[0.8407007455825806, 0.8229171633720398, 0.7699582576751709, 0.7680760622024536, 0.7676540613174438]"
3,106,130,carcinoma of the pancreas,Cancer_dx,None,C3850,carcinoma of the pancreas,1.000000,"[C3850, C3850, C3850, C3850, C3850]","[carcinoma of the pancreas, carcinoma of pancreas, pancreas carcinoma, pancreatic carcinoma, cancer of the pancreas]","[1.000000238418579, 0.9960108995437622, 0.956427276134491, 0.9453885555267334, 0.9308969974517822]"
4,172,194,neoplastic infiltration,Invasion,0.63014996,C187950,tumor infiltration pattern,0.877209,"[C187950, C187950, C48607, C137710, C4194]","[tumor infiltration pattern, tumor infiltration pattern assessment, infiltrating melanoma, infiltrative tumor margin present, infiltrating ductal adenocarcinoma]","[0.8772085905075073, 0.8395975828170776, 0.8199583292007446, 0.8191666603088379, 0.8172265291213989]"
5,222,231,perineural,Site_Other_Body_Part,0.8878,C38293,perineural use,0.914298,"[C38293, C66845, C41407, C38293, C41416]","[perineural use, perineural mpnst, perineural tissue, perineural route of administration, perineural cell]","[0.9142981767654419, 0.8962826132774353, 0.8755375146865845, 0.8609194755554199, 0.8605987429618835]"
6,233,240,invasion,Invasion,0.9605,C20625,invasion,1.000000,"[C20625, C14159, C20625, C75004, C150571]","[invasion, invasive, tumor invasion, cell invasion, muscle invasion]","[1.000000238418579, 0.848335862159729, 0.8024402856826782, 0.7883122563362122, 0.7553536295890808]"
7,246,255,lymph node,Site_Lymph_Node,0.88265,C33769,thoracic lymph node,0.897823,"[C33769, C32298, C77643, C176317, C12358]","[thoracic lymph node, cervical lymph node, aortic lymph node, colon lymph node, head and neck lymph node]","[0.8978233933448792, 0.8929641246795654, 0.889636754989624, 0.8832467198371887, 0.8809458017349243]"
8,257,266,metastasis,Metastasis,0.9999,C40557,metastases,0.938635,"[C40557, C7511, C3577, C3577, C36263]","[metastases, breast metastasis, pulmonary metastasis, lung metastasis, metastatic cancer]","[0.9386346936225891, 0.897005021572113, 0.887118935585022, 0.8848515152931213, 0.8798297047615051]"


### JSON Lines

In [11]:
def create_jsonl(records):
    if isinstance(records, str):
        records = [records]
    json_records = [{"text": text} for text in records]
    json_lines = "\n".join(json.dumps(record) for record in json_records)
    return json_lines

In [12]:
input_jsonl_data = create_jsonl(sample_text)
data = invoke_realtime_endpoint(input_jsonl_data, content_type="application/jsonlines" , accept="application/jsonlines" )
print(data)

{"predictions": [{"begin": 56, "end": 69, "ner_chunk": "adenocarcinoma", "ner_label": "Cancer_dx", "ner_confidence": null, "concept_code": "C5105", "resolution": "colorectal adenocarcinoma", "score": 0.9034679532051086, "all_codes": ["C5105", "C4349", "C4349", "C4349", "C3512"], "all_resolutions": ["colorectal adenocarcinoma", "colon adenocarcinoma", "adenocarcinoma of the colon", "adenocarcinoma of colon", "lung adenocarcinoma"], "all_score": [0.9034679532051086, 0.894524097442627, 0.8823219537734985, 0.8813044428825378, 0.8766085505485535]}, {"begin": 72, "end": 92, "ner_chunk": "acinar cell carcinoma", "ner_label": "Cancer_dx", "ner_confidence": null, "concept_code": "C3768", "resolution": "acinar cell carcinoma", "score": 1.0000001192092896, "all_codes": ["C3768", "C3768", "C3768", "C4197", "C4197"], "all_resolutions": ["acinar cell carcinoma", "acinar carcinoma", "acinar cell adenocarcinoma", "acinar cell neoplasms", "acinar cell neoplasm"], "all_score": [1.0000001192092896, 0.974

### B. Delete the endpoint

Now that you have successfully performed a real-time inference, you do not need the endpoint any more. You can terminate the endpoint to avoid being charged.

In [14]:
model.sagemaker_session.delete_endpoint(model_name)
model.sagemaker_session.delete_endpoint_config(model_name)

## 4. Batch inference

In [15]:
validation_json_file_name = "input.json"
validation_jsonl_file_name = "input.jsonl"

validation_input_json_path = f"s3://{s3_bucket}/{model_name}/validation-input/json/"
validation_output_json_path = f"s3://{s3_bucket}/{model_name}/validation-output/json/"

validation_input_jsonl_path = f"s3://{s3_bucket}/{model_name}/validation-input/jsonl/"
validation_output_jsonl_path = f"s3://{s3_bucket}/{model_name}/validation-output/jsonl/"

def upload_to_s3(input_data, file_name):
    file_format = os.path.splitext(file_name)[1].lower()
    s3_client.put_object(
        Bucket=s3_bucket,
        Key=f"{model_name}/validation-input/{file_format[1:]}/{file_name}",
        Body=input_data.encode("UTF-8"),
    )

In [16]:
# Create JSON and JSON Lines data
input_jsonl_data = create_jsonl(docs)
input_json_data = json.dumps({"text": docs})

# Upload JSON and JSON Lines data to S3
upload_to_s3(input_json_data, validation_json_file_name)
upload_to_s3(input_jsonl_data, validation_jsonl_file_name)

### JSON

In [ ]:
transformer = model.transformer(
    instance_count=1,
    instance_type=batch_transform_inference_instance_type,
    accept="application/json",
    output_path=validation_output_json_path
)

transformer.transform(validation_input_json_path, content_type="application/json")
transformer.wait()

In [ ]:
def retrieve_json_output_from_s3(validation_file_name):
    parsed_url = urlparse(transformer.output_path)
    file_key = f"{parsed_url.path[1:]}{validation_file_name}.out"
    response = s3_client.get_object(Bucket=s3_bucket, Key=file_key)

    data = json.loads(response["Body"].read().decode("utf-8"))
    display(data)

In [20]:
retrieve_json_output_from_s3(validation_json_file_name)

{'predictions': [[{'begin': 37,
    'end': 53,
    'ner_chunk': 'debulking surgery',
    'ner_label': 'Cancer_Surgery',
    'ner_confidence': '0.78400004',
    'concept_code': 'C15749',
    'resolution': 'debulking surgery',
    'score': 0.9999998211860657,
    'all_codes': ['C15749', 'C160865', 'C160866', 'C15749', 'C15749'],
    'all_resolutions': ['debulking surgery',
     'primary debulking surgery',
     'interval debulking surgery',
     'debulking procedure',
     'debulking'],
    'all_score': [0.9999998211860657,
     0.9072895646095276,
     0.8973025679588318,
     0.8779721260070801,
     0.851721465587616]},
   {'begin': 56,
    'end': 77,
    'ner_chunk': 'bilateral oophorectomy',
    'ner_label': 'Cancer_Surgery',
    'ner_confidence': '0.83615005',
    'concept_code': 'C51590',
    'resolution': 'bilateral oophorectomy',
    'score': 0.9999991655349731,
    'all_codes': ['C51590', 'C51765', 'C51765', 'C49133', 'C51667'],
    'all_resolutions': ['bilateral oophorectomy',

### JSON Lines

In [ ]:
transformer = model.transformer(
    instance_count=1,
    instance_type=batch_transform_inference_instance_type,
    accept="application/jsonlines",
    output_path=validation_output_jsonl_path
)
transformer.transform(validation_input_jsonl_path, content_type="application/jsonlines")
transformer.wait()

In [ ]:
def retrieve_jsonlines_output_from_s3(validation_file_name):

    parsed_url = urlparse(transformer.output_path)
    file_key = f"{parsed_url.path[1:]}{validation_file_name}.out"
    response = s3_client.get_object(Bucket=s3_bucket, Key=file_key)

    data = response["Body"].read().decode("utf-8")
    print(data)

In [23]:
retrieve_jsonlines_output_from_s3(validation_jsonl_file_name)

{"predictions": [{"begin": 37, "end": 53, "ner_chunk": "debulking surgery", "ner_label": "Cancer_Surgery", "ner_confidence": "0.78400004", "concept_code": "C15749", "resolution": "debulking surgery", "score": 0.9999998211860657, "all_codes": ["C15749", "C160865", "C160866", "C15749", "C15749"], "all_resolutions": ["debulking surgery", "primary debulking surgery", "interval debulking surgery", "debulking procedure", "debulking"], "all_score": [0.9999998211860657, 0.9072895646095276, 0.8973025679588318, 0.8779721260070801, 0.851721465587616]}, {"begin": 56, "end": 77, "ner_chunk": "bilateral oophorectomy", "ner_label": "Cancer_Surgery", "ner_confidence": "0.83615005", "concept_code": "C51590", "resolution": "bilateral oophorectomy", "score": 0.9999991655349731, "all_codes": ["C51590", "C51765", "C51765", "C49133", "C51667"], "all_resolutions": ["bilateral oophorectomy", "bilateral salpingo-oophorectomy", "bilateral salpingectomy with oophorectomy", "bilateral prophylactic oophorectomy", 

In [24]:
model.delete_model()

INFO:sagemaker:Deleting model with name: ncit-vdb-resolver-2-2025-08-18-12-52-01-495


### Unsubscribe to the listing (optional)

If you would like to unsubscribe to the model package, follow these steps. Before you cancel the subscription, ensure that you do not have any [deployable model](https://console.aws.amazon.com/sagemaker/home#/models) created from the model package or using the algorithm. Note - You can find this information by looking at the container name associated with the model. 

**Steps to unsubscribe to product from AWS Marketplace**:
1. Navigate to __Machine Learning__ tab on [__Your Software subscriptions page__](https://aws.amazon.com/marketplace/ai/library?productType=ml&ref_=mlmp_gitdemo_indust)
2. Locate the listing that you want to cancel the subscription for, and then choose __Cancel Subscription__  to cancel the subscription.

